# Patterngenerator

> Creates from a pattern-string with placeholders possible subpaterns that matches possible placement spaces for a a crossword generator

In [ ]:
#| default_exp pattern_generator

In [ ]:
#| export
from fastcore.foundation import L

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| export
class PatternGenerator:
    def __init__(self, slot_pattern, min_word_length=2, space_char=' '):
        self.slot_pattern = slot_pattern
        self.min_word_length = min_word_length
        self.space_char = space_char
    
    def generate_patterns(self):
        # Returns sorted list of (offset, pattern)
        patterns = list(self._extract_subpatterns())
        patterns.sort(key=lambda x: x[2], reverse=True)  # Sort by sort_key
        return list(L(patterns).map(lambda x: x[:2]))
    
    def _extract_subpatterns(self):
        only_spaces = all([self.space_char == x for x in self.slot_pattern.split()])
        patterns = L()
        
        for i in range(len(self.slot_pattern) + 1 - self.min_word_length):
            for j in range(i + self.min_word_length, len(self.slot_pattern) + 1):
                part_pat = self.slot_pattern[i:j]
                
                if (i == 0 or self.slot_pattern[i-1] == self.space_char) and \
                   (j == len(self.slot_pattern) or self.slot_pattern[j] == self.space_char):
                    sort_key = self._make_sort_key(part_pat, i)
                    patterns.append((i, part_pat, sort_key))
        
        if not only_spaces:
            patterns = patterns.filter(lambda y: not all([self.space_char == x for x in y[1].split()]))
        return set(patterns)
    
    def _make_sort_key(self, part_pattern, offset):
        pos_t = self._get_position_type(offset, len(part_pattern))
        cross_count = self._count_constraints(part_pattern)
        pat_len = len(part_pattern)
        return "%s_%03x_%03x" % (pos_t, cross_count, pat_len)
    
    def _count_constraints(self, pattern):
        return len(pattern.replace(self.space_char, ''))
    
    def _get_position_type(self, offset, pattern_length):
        if offset == 0:
            return 's'
        if offset + pattern_length == len(self.slot_pattern):
            return 'l'
        return 'i'

In [ ]:
# Test with slot variable
slot = '  x ch  p'
pg = PatternGenerator(slot)
result = pg.generate_patterns()
result_patterns = set([x[1] for x in result])

assert '  x ch  p' in result_patterns
assert '  x' in result_patterns
assert '  x ch' in result_patterns
assert '  x ch ' in result_patterns
assert ' x' in result_patterns
assert ' x ch' in result_patterns
assert ' x ch ' in result_patterns
assert ' x ch  p' in result_patterns
assert 'x ch' in result_patterns
assert 'x ch ' in result_patterns
assert 'x ch  p' in result_patterns
assert 'ch' in result_patterns
assert 'ch ' in result_patterns
assert 'ch  p' in result_patterns
assert ' p' in result_patterns

# Negative tests
pg1 = PatternGenerator('x', min_word_length=2)
result1 = set([x[1] for x in pg1.generate_patterns()])
assert len(result1) == 0

pg2 = PatternGenerator('', min_word_length=2)
result2 = set([x[1] for x in pg2.generate_patterns()])
assert len(result2) == 0

pg3 = PatternGenerator('    ', min_word_length=2)
result3 = set([x[1] for x in pg3.generate_patterns()])
assert all(' ' in p for p in result3)

pg4 = PatternGenerator('abc', min_word_length=2)
result4 = set([x[1] for x in pg4.generate_patterns()])
assert len(result4) == 1

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()